# Calendar generator

This script generates a calendar to be imported primarily to Google Calendar. Main features:

- List the matches of the selected competition and team
- Generates an output
- played matches has a score. It is also displayed.

In [1]:
#import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import arrow
from datetime import datetime
from ics import Calendar, Event

# Setting up parameters

In [2]:
#match_id = '76648'
#match_id = '75806'
#match_url = 'http://www.waterpolo.hu/meccs/' + match_id

competition_id = '680' # Fiú országos bajnokság
#competition_id = '704' # Gyermek I. Budapest bajnokság
competition_id = '768' # 2023-24 Bp serdülő (régi ID)

base_url = 'https://waterpolo.hu'
base_url = 'http://mvlsz.webpont.com'

competition_url = 'https://waterpolo.hu/bajnoksagok/?szures[bajnoksag_id]=' + competition_id
competition_url = 'http://mvlsz.webpont.com/bajnoksagok/?szures[bajnoksag_id]=' + competition_id

selected_team = "KSI"

# Parsing competition using BeutifulSoup

In [3]:
page = requests.get(competition_url, timeout=10)
#soup = BeautifulSoup(page.content, 'html-parser')
soup = BeautifulSoup(page.content, 'html5lib')
matches = soup.find_all("tr", class_='gamerow')


In [4]:
competition = list()

for match in matches:
    mtch = {'teams': []}
    competition.append(mtch)
    mtch['rawdata'] = mtch
    location = match.td
    for i, row in enumerate(match.find_all('td')):
        if row.text.startswith("202"):
            mtch['date_str'] = row.text.strip()
        elif i == 6 and row.text.strip() != '':
            mtch['location'] = row.text.strip()
    for link in match.find_all("a"):
        href = link['href']
        if '/csapat/'  in href:
            mtch['teams'].append(link.text)
        elif '/MatchReturnData' in href:
            mtch['report_url'] = href
        elif '/meccs/' in href:
            mtch['match_url'] = href
            mtch['result'] = link.text
        elif 'www.google.com/calendar/event' in href:
            mtch['google_calendar'] = href
        else:
            print(' nem feldolgozott sor: ', link)
random.sample(competition, 5)

[{'teams': [], 'rawdata': {...}},
 {'teams': ['Honvéd', 'FTC-Telekom'],
  'rawdata': {...},
  'date_str': '2024. ápr. 27. 00:00',
  'location': 'NINCS pálya megadva',
  'google_calendar': 'http://www.google.com/calendar/event?action=TEMPLATE&text=Budapesti+Ifj%C3%BAs%C3%A1gi+Fi%C3%BA%3A+Honv%C3%A9d+-+FTC-Telekom&dates=20240427T000000/20240427T013000&sprop=Weboldal:http://waterpolo.hu&details=Budapesti+Ifj%C3%BAs%C3%A1gi+Fi%C3%BA+Bajnoks%C3%A1g+%282005%29%0AAlapszakasz+-+1.+csoport++-+19.+fordul%C3%B3++-+2024-04-27+00%3A00%0AHonv%C3%A9d+-+FTC-Telekom+%28NINCS+p%C3%A1lya+megadva%29%0AJ%C3%A1t%C3%A9kvezet%C5%91k%3A+&location=NINCS+p%C3%A1lya+megadva&trp=true',
  'match_url': '/meccs/81928',
  'result': '0-0',
  'report_url': '/store_pdf/MatchReturnData_768_81928.pdf'},
 {'teams': ['Genesys OSC Újbuda11', 'KSI SE'],
  'rawdata': {...},
  'date_str': '2024. ápr. 06. 00:00',
  'location': 'NINCS pálya megadva',
  'google_calendar': 'http://www.google.com/calendar/event?action=TEMPLATE&text=B

## Cleaning data

In [5]:
for match in competition:
    if len(match) < 5:
        del match

def filter_competition_by_team(competition: list, team: str) -> list:
    filtered = list()
    for match in competition:
        if len(match) > 3:
            for t in match['teams']:
                if team in t:
                    filtered.append(match)
                    break
    return filtered

competition_ksi = filter_competition_by_team(competition, "KSI")

for i, match in enumerate(competition_ksi):
    print(i, match['date_str'], match['teams'], match['result'] if match['result'] != '0-0' else '')


0 2023. szept. 23. 09:00 ['KSI SE', 'FTC-Telekom'] 7-5
1 2023. szept. 23. 12:00 ['KSI SE', 'Vasas SC'] 
2 2023. okt. 14. 00:00 ['Genesys OSC Újbuda11', 'KSI SE'] 
3 2023. okt. 14. 00:00 ['KSI SE', 'Genesys OSC Újbuda'] 
4 2023. nov. 04. 00:00 ['KSI SE', 'Honvéd'] 
5 2023. nov. 25. 00:00 ['BVSC-Zugló', 'KSI SE'] 
6 2023. nov. 25. 00:00 ['FTC-Telekom', 'KSI SE'] 
7 2023. dec. 16. 00:00 ['Vasas SC', 'KSI SE'] 
8 2023. dec. 16. 00:00 ['KSI SE', 'Genesys OSC Újbuda11'] 
9 2024. jan. 27. 00:00 ['Genesys OSC Újbuda', 'KSI SE'] 
10 2024. febr. 17. 00:00 ['Honvéd', 'KSI SE'] 
11 2024. febr. 17. 00:00 ['KSI SE', 'BVSC-Zugló'] 
12 2024. márc. 09. 00:00 ['KSI SE', 'FTC-Telekom'] 
13 2024. márc. 09. 00:00 ['KSI SE', 'Vasas SC'] 
14 2024. ápr. 06. 00:00 ['Genesys OSC Újbuda11', 'KSI SE'] 
15 2024. ápr. 06. 00:00 ['KSI SE', 'Genesys OSC Újbuda'] 
16 2024. máj. 11. 00:00 ['KSI SE', 'Honvéd'] 
17 2024. máj. 11. 00:00 ['BVSC-Zugló', 'KSI SE'] 


## Generating calendar

In [6]:
calendar = Calendar()

for match in competition_ksi:
    event = Event()
    result = (': ' + match['result']) if match['result'] != '0-0' else ''
    event.name = f'{match["teams"][0]} - {match["teams"][1]}{result}'
    event.description = f'<a href="{base_url}{match["match_url"]}">Adatlap</a>\n<a href="{competition_url}">Bajnokság</a>'
    event.begin = arrow.get(match['date_str'], "YYYY. MMM. D. H:mm", locale="hu", tzinfo="CET")
    event.duration = {'hours': 1}
    event.last_modified = datetime.now()
    event.location = match['location']
    calendar.events.add(event)


print(calendar.serialize())

BEGIN:VCALENDAR
VERSION:2.0
PRODID:ics.py - http://git.io/lLljaA
BEGIN:VEVENT
DESCRIPTION:<a href="http://mvlsz.webpont.com/meccs/81922">Adatlap</a>\n<a href="http://mvlsz.webpont.com/bajnoksagok/?szures[bajnoksag_id]=768">Bajnokság</a>
DURATION:PT1H
LAST-MODIFIED:20230923T082651Z
LOCATION:NINCS pálya megadva
DTSTART:20240405T220000Z
SUMMARY:Genesys OSC Újbuda11 - KSI SE
UID:1768fdf8-199b-4f29-ba84-f9d57bb778f1@1768.org
END:VEVENT
BEGIN:VEVENT
DESCRIPTION:<a href="http://mvlsz.webpont.com/meccs/81876">Adatlap</a>\n<a href="http://mvlsz.webpont.com/bajnoksagok/?szures[bajnoksag_id]=768">Bajnokság</a>
DURATION:PT1H
LAST-MODIFIED:20230923T082651Z
LOCATION:Széchy T. u. Margitsz.
DTSTART:20230923T070000Z
SUMMARY:KSI SE - FTC-Telekom: 7-5
UID:1f93c5b8-bf00-4ef2-8174-006f9345d662@1f93.org
END:VEVENT
BEGIN:VEVENT
DESCRIPTION:<a href="http://mvlsz.webpont.com/meccs/81914">Adatlap</a>\n<a href="http://mvlsz.webpont.com/bajnoksagok/?szures[bajnoksag_id]=768">Bajnokság</a>
DURATION:PT1H
LAST-MODIF